In [ ]:
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
import numpy as np
import matplotlib.pyplot as plt
import torch.utils.data as data

from torch.optim.lr_scheduler import _LRScheduler

from sklearn import decomposition
from sklearn import manifold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

import copy
from collections import namedtuple
import os
import random
import shutil
import time
import pandas as pd


SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
# Check whether GPU is available and if it is then use GPU
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

In [ ]:
# Data augmentation
train2_transforms = transforms.Compose([transforms.Resize((256,256)),
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                            std=[0.229, 0.224, 0.225])])

# Use 10-crop for Test Time Augmentation
test_transforms = transforms.Compose([transforms.Resize(256),
                                      transforms.TenCrop(224),
                                      transforms.Lambda(lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops])),
                                      transforms.Lambda(lambda crops: torch.stack([transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])(crop) for crop in crops]))])

# Load the datasets with ImageFolder
train_data = datasets.ImageFolder("dataset/train", transform=train2_transforms)
valid_data = datasets.ImageFolder("dataset/valid", transform=test_transforms)
test_data = datasets.ImageFolder("dataset/test", transform=test_transforms)

# Using the image datasets and the tranforms, define the dataloaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size = 50, shuffle = True)
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size = 50, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size= 50, shuffle = True)

classes=os.listdir('dataset/train')

In [ ]:
# Load the ResNet-50 model pretraned on ImageNet 
model = models.resnet50(pretrained=True)
classifier = nn.Linear(2048, 2)
model.fc = classifier
print(model)
criterion = nn.CrossEntropyLoss()
FOUND_LR = 1e-3

params = [
          {'params': model.conv1.parameters(), 'lr': FOUND_LR / 20},
          {'params': model.bn1.parameters(), 'lr': FOUND_LR / 10},
          {'params': model.layer1.parameters(), 'lr': FOUND_LR / 8},
          {'params': model.layer2.parameters(), 'lr': FOUND_LR / 6},
          {'params': model.layer3.parameters(), 'lr': FOUND_LR / 4},
          {'params': model.layer4.parameters(), 'lr': FOUND_LR / 2},
          {'params': model.fc.parameters(), 'lr': FOUND_LR *10}
         ]


optimizer = optim.SGD(params, lr = FOUND_LR)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience = 5)

In [ ]:
model1=model
model1.fc=nn.Identity()
from torchsummary import summary
summary(model1, (3, 224, 224), device='cuda')

In [ ]:
# Move the model to GPU if CUDA is available
temp=list()
x=1
if train_on_gpu:
    model1.cuda()

    model1.train()
    for data, target in train_loader:
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model1(data)
        outputt = np.squeeze(output.cpu().detach().numpy())
        print(x)
        if x==1:
          fitur=pd.DataFrame(outputt)
          x=2
        else:
          fitur=pd.concat([fitur,pd.DataFrame(outputt)])
        temp=np.concatenate((temp,np.squeeze(target.cpu().detach().numpy())))
for i in range(len(temp)):
  temp[i]=int(temp[i])
featurestrain=pd.concat([fitur.reset_index(drop=True),pd.DataFrame(temp).reset_index(drop=True)],ignore_index=True, axis=1)

In [ ]:
# Move the model to GPU if CUDA is available
temp=list()
x=1
if train_on_gpu:
    model1.cuda()

    model1.train()
    for data, target in valid_loader:
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model1(data)
        outputt = np.squeeze(output.cpu().detach().numpy())
        print(x)
        if x==1:
          fitur=pd.DataFrame(outputt)
          x=2
        else:
          fitur=pd.concat([fitur,pd.DataFrame(outputt)])
        temp=np.concatenate((temp,np.squeeze(target.cpu().detach().numpy())))
for i in range(len(temp)):
  temp[i]=int(temp[i])
featuresvalid=pd.concat([fitur.reset_index(drop=True),pd.DataFrame(temp).reset_index(drop=True)],ignore_index=True, axis=1)

In [ ]:
# Move the model to GPU if CUDA is available
temp=list()
x=1
if train_on_gpu:
    model1.cuda()

    model1.train()
    for data, target in test_loader:
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model1(data)
        outputt = np.squeeze(output.cpu().detach().numpy())
        print(x)
        if x==1:
          fitur=pd.DataFrame(outputt)
          x=2
        else:
          fitur=pd.concat([fitur,pd.DataFrame(outputt)])
        temp=np.concatenate((temp,np.squeeze(target.cpu().detach().numpy())))
for i in range(len(temp)):
  temp[i]=int(temp[i])
featurestest=pd.concat([fitur.reset_index(drop=True),pd.DataFrame(temp).reset_index(drop=True)],ignore_index=True, axis=1)

In [ ]:
featurestrainfinal=pd.concat([featurestrain, featuresvalid, featurestest])
print(featurestest.shape)
print(featurestrain.shape)
print(featuresvalid.shape)
featurestrain.to_csv('featurestrain.csv', index=False)
featurestest.to_csv('featurestest.csv', index=False)
featuresvalid.to_csv('featuresvalid.csv', index=False)
featurestrainfinal.to_csv('featuresfinal.csv', index=False)